In [6]:
import pandas as pd
import numpy as np
import math
import itertools
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1.inset_locator import (inset_axes, InsetPosition, mark_inset)
import math
import copy
import re
from collections import OrderedDict 
import Bio
from Bio import SeqIO
import xls_analysis_classes
from xls_analysis_classes import *

#from xls_analysis_classes import *

matplotlib.rcParams.update({'font.size': 10})

In [7]:
d = pd.read_csv('./proteasome-tri-crosslinks.csv', sep = ',')
data = pd.DataFrame(d)
#data.insert(2, 'dummy A', 'prot1')
#data.insert(3, 'dummy XLA', 'K1')
data.insert(2, 'Subunit DB', 'prot1')
data.insert(3, 'XL DB', 'K1')
#
data.insert(6, 'dummy B', 'prot1')
data.insert(7, 'dummy XLB', 'K1')
data.insert(10, 'dummy C', 'prot1')
data.insert(11, 'dummy XLC', 'K1')
#print(data)
df1 = data.iloc[:, :4].join(data.iloc[:, 12:13]).join(data.iloc[:, 15:16]).join(data.iloc[:, 18:32]).join(data.iloc[:, 46:53])
df1.insert(6, 'Common/Gene Name B', 'prot1')
print(df1.columns)
df2 = data.iloc[:, 4:8].join(data.iloc[:, 13:14]).join(data.iloc[:, 16:17]).join(data.iloc[:, 18:25]).join(data.iloc[:, 32:39]).join(data.iloc[:, 46:53])
#print(df2.columns)
df3 = data.iloc[:, 8:12].join(data.iloc[:, 14:15]).join(data.iloc[:, 17:18]).join(data.iloc[:, 18:25]).join(data.iloc[:, 39:46]).join(data.iloc[:, 46:53])
#print(df3.columns)
df1.to_csv('proteasome-tri-crosslinks-1.csv', index = False)
df2.to_csv('proteasome-tri-crosslinks-2.csv', index = False)
df3.to_csv('proteasome-tri-crosslinks-3.csv', index = False)
#print(df3)

Index(['Subunit A', 'XL A', 'Subunit DB', 'XL DB', 'Subunit Name A',
       'Common/Gene Name A', 'Common/Gene Name B', 'XL Scan', 'XL RT',
       'XL m/z', 'XL Charge', 'XL PPM', 'Corr. XL PPM', 'Corr. XL Isotope',
       'm/z A', 'Charge A', 'Sequence A', 'Protein Mods A', 'PPM A', 'Score A',
       'EV A', 'Ambiguous', 'Redundant', 'Subunit Names', 'Linkage Info',
       'Redundant Count', 'Unique Count', 'CLMS-CSV'],
      dtype='object')


In [50]:
# open a file which is down two directories and called "definitions.txt" and read it
# open this file as a pandas dataframe and name the columns as genename, genericname
heads = ['genename', 'genericname', 'chain1', 'chain2']
dnames = pd.read_csv('../../definitions.txt', names = heads)
print(dnames)
# define a function that replaces or inserts the chainid1 based on the gene name
def set_chain_id1(gene):
    heads = ['genename', 'genericname', 'chain1', 'chain2']
    dnames = pd.read_csv('../../definitions.txt', names = heads)
    for i in range(len(dnames['genename'])):
        if gene == dnames['genename'][i]:
            return dnames['chain1'][i] 
# Define a function to set the chain ID based on the gene value
def set_chain_id(gene):
    if gene == 'Rpt2':
        return 'I'
    elif gene == 'Rpt4':
        return 'L'
    elif gene == 'Rpt5':
        return 'M'
    elif gene == 'Rpt6': 
        return 'J'
    elif gene == 'Rpt1':
        return 'H'
    elif gene == 'Rpt3':
        return 'K'
d['Chain ID A'] = d['Common/Gene Name A'].apply(set_chain_id1)
d['Chain ID B'] = d['Common/Gene Name B'].apply(set_chain_id1)
d['Chain ID C'] = d['Common/Gene Name C'].apply(set_chain_id1)
print(d['Chain ID A'].dropna())
#print(d.columns)

   genename  genericname chain1 chain2
0      Rpt3           6B      K      y
1      Rpt4          10B      L      z
2      Rpt6            8      J      x
3      Rpt2            4      I      w
4      Rpt5           6A      0      M
5      Rpt1            7      H      v
6      Rpn6    non-ATP 2     AC      Z
7     Rpn11   non-ATP 14      9      V
8      Rpn7    non-ATP 6      5      R
9      Rpn3    non-ATP 3      6      S
10     Rpn8    non-ATP 7      8      U
11     Rpn2    non-ATP 1      1     N 
12     Rpn5   non-ATP 12      3      P
13     Rpn9   non-ATP 13      2      O
Index(['Subunit A', 'XL A', 'Subunit B', 'XL B', 'Subunit C', 'XL C',
       'Subunit Name A', 'Subunit Name B', 'Subunit Name C',
       'Common/Gene Name A', 'Common/Gene Name B', 'Common/Gene Name C',
       'XL Scan', 'XL RT', 'XL m/z', 'XL Charge', 'XL PPM', 'Corr. XL PPM',
       'Corr. XL Isotope', 'm/z A', 'Charge A', 'Sequence A', 'Protein Mods A',
       'PPM A', 'Score A', 'EV A', 'm/z B', 'Charge B',